In [0]:
h2o.shutdown(prompt=False)

[WARNING] in <ipython-input-68-02ff3806b010> line 1:
    >>> h2o.shutdown(prompt=False)
        ^^^^ Deprecated, use ``h2o.cluster().shutdown()``.
Exception ignored in:<bound method ExprNode.__del__ of <Expr(h2o.runif <Expr(as.numeric <Expr()#data.hex>)#py_21_sid_b4c3; scalar> -1)#py_24_sid_b4c3; scalar>>
Traceback (most recent call last):
  File "/home/dfernandez/.local/lib/python3.6/site-packages/h2o/expr.py", line 187, in __del__
ExprNode.rapids("(rm {})".format(self._cache._id))
  File "/home/dfernandez/.local/lib/python3.6/site-packages/h2o/expr.py", line 241, in rapids
return h2o.api("POST /99/Rapids", data={"ast": expr, "session_id": h2o.connection().session_id})
  File "/home/dfernandez/.local/lib/python3.6/site-packages/h2o/h2o.py", line 104, in api
return h2oconn.request(endpoint, data=data, json=json, filename=filename, save_to=save_to)
  File "/home/dfernandez/.local/lib/python3.6/site-packages/h2o/backend/connection.py", line 434, in request
return self._process_response(r

In [3]:
!pip3 install h2o

     |████████████████████████████████| 123.7MB 86kB/s 
  Created wheel for h2o: filename=h2o-3.26.0.11-py2.py3-none-any.whl size=123721708 sha256=0552d4b76906f184d8307360031bee85b4a95c1351f87f9b94190ffe62a25fb4
  Stored in directory: /root/.cache/pip/wheels/71/e6/ed/fb9be80ae2eb0bef9deada74fc370b2532f578d8b00bc1a8fb
Successfully built h2o


In [1]:
import h2o
import numpy as np
import pandas as pd

In [2]:
h2o.init(nthreads = -1, max_mem_size = 4)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.4" 2019-07-16; OpenJDK Runtime Environment (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3); OpenJDK 64-Bit Server VM (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3, mixed mode, sharing)
  Starting server from /home/dfernandez/.local/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpmuvndwht
  JVM stdout: /tmp/tmpmuvndwht/h2o_dfernandez_started_from_python.out
  JVM stderr: /tmp/tmpmuvndwht/h2o_dfernandez_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,Europe/Madrid
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.11
H2O cluster version age:,4 days
H2O cluster name:,H2O_from_python_dfernandez_zv2h92
H2O cluster total nodes:,1
H2O cluster free memory:,4 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


In [3]:
import pandas as pd 
df_pandas=pd.read_csv('Input/data.csv')
df=h2o.H2OFrame(df_pandas)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [4]:
df.head()

carat,cut,color,clarity,depth,table,x,y,z,price
2.26,Ideal,G,SI2,61.9,57,8.44,8.36,5.2,12831
2.43,Very Good,H,SI2,63.2,57,8.56,8.5,5.39,16170
0.8,Premium,F,SI2,61,57,6.03,6.01,3.67,2797
0.4,Ideal,F,I1,63.3,60,4.68,4.64,2.95,630
0.31,Ideal,G,VS2,61.6,55,4.39,4.37,2.7,698
0.53,Very Good,F,SI2,60.5,60,5.19,5.23,3.15,1132
0.7,Premium,D,SI2,60.8,61,5.69,5.64,3.45,1987
1.13,Good,F,SI2,64,58,6.52,6.58,4.19,3952
0.35,Ideal,E,SI1,60.9,55,4.53,4.57,2.77,614
1.01,Fair,H,VS1,61.4,66,6.34,6.24,3.87,5062


In [5]:
df.types

{'carat': 'real',
 'cut': 'enum',
 'color': 'enum',
 'clarity': 'enum',
 'depth': 'real',
 'table': 'real',
 'x': 'real',
 'y': 'real',
 'z': 'real',
 'price': 'int'}

In [6]:
df = df.asnumeric()

In [7]:
df.types

{'carat': 'real',
 'cut': 'real',
 'color': 'real',
 'clarity': 'real',
 'depth': 'real',
 'table': 'real',
 'x': 'real',
 'y': 'real',
 'z': 'real',
 'price': 'real'}

In [8]:
df.head()

carat,cut,color,clarity,depth,table,x,y,z,price
2.26,2,3,3,61.9,57,8.44,8.36,5.2,12831
2.43,4,4,3,63.2,57,8.56,8.5,5.39,16170
0.8,3,2,3,61,57,6.03,6.01,3.67,2797
0.4,2,2,0,63.3,60,4.68,4.64,2.95,630
0.31,2,3,5,61.6,55,4.39,4.37,2.7,698
0.53,4,2,3,60.5,60,5.19,5.23,3.15,1132
0.7,3,0,3,60.8,61,5.69,5.64,3.45,1987
1.13,1,2,3,64,58,6.52,6.58,4.19,3952
0.35,2,1,2,60.9,55,4.53,4.57,2.77,614
1.01,0,4,4,61.4,66,6.34,6.24,3.87,5062


In [9]:
y_columns = "price"
x_columns = ["carat","cut","color", "clarity","depth","table","x","y","z"]

In [11]:
train, test = df.split_frame()
X_train=train[x_columns]
y_train=train[y_columns]
X_test=test[x_columns]
y_test=test[y_columns]

In [12]:
test

carat,cut,color,clarity,depth,table,x,y,z,price
0.8,3,2,3,61,57,6.03,6.01,3.67,2797
0.7,3,0,3,60.8,61,5.69,5.64,3.45,1987
0.3,3,0,2,59.7,59,4.34,4.37,2.6,552
1.23,2,5,3,61.2,58,6.86,6.9,4.21,5315
0.51,3,1,7,61,61,5.17,5.13,3.14,2060
1.57,3,4,5,61,59,7.5,7.54,4.59,11470
1.02,4,1,3,63,57,6.39,6.44,4.04,4398
0.32,2,2,7,62.1,56,4.4,4.46,2.75,741
0.34,2,3,5,62.2,53,4.46,4.51,2.79,596
0.52,2,3,4,62.3,55,5.14,5.16,3.21,1689


In [13]:
from h2o.estimators.random_forest import H2ORandomForestEstimator as rf

In [13]:
rf_fit1 = rf(model_id='rf_fit1', seed=1)
rf_fit1.train(x=x_columns, y=y_columns, training_frame=train)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [14]:
rf_fit1.model_performance

Model Details
H2ORandomForestEstimator:Distributed Random Forest
Model Key:rf_fit1


Model Summary:


ModelMetricsRegression: drf
** Reported on train data. **

MSE: 372781.9576249033
RMSE: 610.5587257790222
MAE: 304.3938680421208
RMSLE: 0.10396570897800318
Mean Residual Deviance: 372781.9576249033

Scoring History:

Variable Importances:


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,50.0,50.0,9510333.0,20.0,20.0,20.0,14277.0,15810.0,15161.8


,,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance
0,,2019-12-10 09:24:28,0.150 sec,0.0,NaN,NaN,NaN
1,,2019-12-10 09:24:30,2.658 sec,1.0,932.477639,450.839057,869514.546745
2,,2019-12-10 09:24:31,3.761 sec,2.0,916.861286,441.883143,840634.618632
3,,2019-12-10 09:24:36,8.060 sec,7.0,784.457360,382.535091,615373.349395
4,,2019-12-10 09:24:40,12.220 sec,24.0,641.393189,319.724403,411385.222813
5,,2019-12-10 09:24:44,16.313 sec,40.0,619.724000,308.845606,384057.835725
6,,2019-12-10 09:24:47,19.121 sec,50.0,610.558726,304.393868,372781.957625


,variable,relative_importance,scaled_importance,percentage
0,y,6.344021e+12,1.000000,0.298675
1,carat,5.884808e+12,0.927615,0.277055
2,x,4.445536e+12,0.700744,0.209295
3,z,2.906846e+12,0.458203,0.136854
4,clarity,8.931926e+11,0.140793,0.042051
5,color,5.266694e+11,0.083018,0.024795
6,depth,1.004913e+11,0.015840,0.004731
7,table,8.922883e+10,0.014065,0.004201
8,cut,4.975086e+10,0.007842,0.002342


<bound method ModelBase.model_performance of >

In [14]:
from h2o.automl import H2OAutoML
aml_ti = H2OAutoML(max_runtime_secs= 180,max_models= 15, seed= 1, nfolds=0)
aml_ti.train(x = x_columns, y = y_columns, training_frame = train, validation_frame=test)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [15]:
lb_ti = aml_ti.leaderboard
lb_ti

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
XGBoost_1_AutoML_20191210_112901,235402,485.183,235402,256.202,0.102319
GBM_1_AutoML_20191210_112901,239794,489.688,239794,264.409,0.105405
XGBoost_2_AutoML_20191210_112901,243449,493.406,243449,254.798,0.0906101
GBM_4_AutoML_20191210_112901,243643,493.602,243643,262.08,nan
GBM_3_AutoML_20191210_112901,246931,496.922,246931,266.115,nan
GBM_2_AutoML_20191210_112901,257494,507.438,257494,271.501,0.107626
XGBoost_3_AutoML_20191210_112901,260023,509.925,260023,277.929,0.113643
GBM_5_AutoML_20191210_112901,268258,517.936,268258,264.819,0.108369
XGBoost_grid_1_AutoML_20191210_112901_model_1,281029,530.121,281029,276.206,0.0974811
XRT_1_AutoML_20191210_112901,303023,550.475,303023,280.832,0.106224


In [12]:
from h2o.automl import H2OAutoML
aml_ti2 = H2OAutoML(max_runtime_secs = 30)
aml_ti2.train(x = x_columns, y = y_columns, training_frame = train, validation_frame=test)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [14]:
lb_ti2 = aml_ti2.leaderboard
lb_ti2

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
XGBoost_1_AutoML_20191210_093418,2.22781e+06,1492.59,2.22781e+06,893.006,0.280172


In [16]:
df2 = pd.read_csv('Input/test.csv')
df2 = h2o.H2OFrame(df2)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [17]:
x = df2.drop('id')

In [18]:
x = x.asnumeric()

In [19]:
preds = aml_ti.leader.predict(x)

xgboost prediction progress: |████████████████████████████████████████████| 100%


In [20]:
preds2 = preds.as_data_frame(use_pandas=True)

In [21]:
preds2

,predict
0,1249.086914
1,6521.639648
2,1673.309204
3,3788.924805
4,1774.441040
...,...
13480,2271.568604
13481,2006.381958
13482,11464.254883
13483,520.657227


In [22]:
test = df2.as_data_frame(use_pandas=True)

In [23]:
results = pd.DataFrame()
results['id'] = test['id']
results['price'] = preds2['predict']

In [24]:
# Create & upload a file.
results.to_csv('./Output/results3.csv',index=False)
